In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
library("stringr")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,613876,32.8,1330087,71.1,1130880,60.4
Vcells,1149379,8.8,8388608,64.0,1801055,13.8


Loading required package: data.table



In [2]:
kdatasetinput <- "c2_reparacNA_fe_inicial.csv.gz"
kdatasetoutput <- "c2_reparacNA_fe_inicialymin.csv.gz"

In [3]:
setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread(paste0("./datasets/",kdatasetinput))   #donde entreno# FIN Parametros del script

In [8]:

AgregarVariables  <- function( dataset )
{
  gc()
 
############# FE 1 ####################
   
  #Aqui debe usted agregar sus propias nuevas variables           
    dataset[ , prop_vida_en_banco          := cliente_antiguedad/(12*cliente_edad)]
       
    #Prestamos
    dataset[ , cprestamos                 := rowSums( cbind( cprestamos_personales, cprestamos_prendarios, cprestamos_hipotecarios) , na.rm=TRUE ) ]
    dataset[ , mprestamos                 := rowSums( cbind( mprestamos_personales, mprestamos_prendarios, mprestamos_hipotecarios) , na.rm=TRUE ) ]
       
    #Comisiones
    dataset[ , ccomisiones                := rowSums( cbind( ccomisiones_mantenimiento,ccomisiones_otras ) , na.rm=TRUE ) ]
    dataset[ , mcomisiones                := rowSums( cbind( mcomisiones_mantenimiento,mcomisiones_otras ) , na.rm=TRUE ) ]
    dataset[ , paga_comisiones             := ifelse(ccomisiones > 0,1,0)]

   
      #Payroll
   dataset[ , cpayroll_total    := rowSums( cbind(cpayroll_trx,cpayroll2_trx ) , na.rm=TRUE ) ]
   dataset[ , mpayroll_total    := rowSums( cbind(mpayroll,mpayroll2 ) , na.rm=TRUE ) ]
    dataset[ , acredita_sueldo  := ifelse(cpayroll_total > 0,1,0)]

   
    #Mes
    dataset[ , mes   := str_sub(foto_mes, start=-2)  ]

     #ctrx_normalizada
    dataset[ , ctrx_quarter_normalizado_log   := log(ctrx_quarter_normalizado+1)  ]
    dataset[ , ctrx_quarter_normalizado_pow2   := ctrx_quarter_normalizado^2  ]

############# FIN FE 1 ####################
    
    
    
    
    
    
    
    
    
    
    
    
    
    
  #valvula de seguridad para evitar valores infinitos
  #paso los infinitos a NULOS
  infinitos      <- lapply(names(dataset),function(.name) dataset[ , sum(is.infinite(get(.name)))])
  infinitos_qty  <- sum( unlist( infinitos) )
  if( infinitos_qty > 0 )
  {
    cat( "ATENCION, hay", infinitos_qty, "valores infinitos en tu dataset. Seran pasados a NA\n" )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  #valvula de seguridad para evitar valores NaN  que es 0/0
  #paso los NaN a 0 , decision polemica si las hay
  #se invita a asignar un valor razonable segun la semantica del campo creado
  nans      <- lapply(names(dataset),function(.name) dataset[ , sum(is.nan(get(.name)))])
  nans_qty  <- sum( unlist( nans) )
  if( nans_qty > 0 )
  {
    cat( "ATENCION, hay", nans_qty, "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n" )
    cat( "Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

}

In [9]:
#primero agrego las variables manuales
AgregarVariables( dataset )

ATENCION, hay 91 valores infinitos en tu dataset. Seran pasados a NA


In [10]:
fwrite( dataset,
        file=paste0("./datasets/",kdatasetoutput),
        sep= "," )

In [11]:
ncol(dataset)

[1] 207

In [12]:
nrow(dataset)

[1] 4214386